In [ ]:
#!pip install xgboost
import xgboost as xgb

!pip install catboost
from catboost import CatBoostClassifier, Pool

!pip install pyhive pandas hdfs
from pyhive import hive

import numpy as np # linear algebra
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import class_weight
from keras.utils import to_categorical
from scipy.stats import randint
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.datasets import make_classification
from sklearn.svm import SVC


from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
import plotly.express as px
init_notebook_mode(connected=True)
import plotly.graph_objs as go


from scipy.stats import randint
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, GRU, Dense, Conv2D, MaxPooling2D, Flatten, LSTM,Conv1D,MaxPool1D,Dropout,Reshape
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


In [ ]:
# Connect to Hive
conn = hive.Connection(host='192.168.1.171', port=10000, username='vboxuser')
query = "SELECT * FROM default.net_data"
df_dataset = pd.read_sql(query, conn)

Mounted at /content/drive


In [ ]:

df_dataset.reset_index(drop=True, inplace=True)
len(df_dataset.columns)

16

In [ ]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068376 entries, 0 to 1068375
Data columns (total 16 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   avg_ipt        1068376 non-null  float64
 1   bytes_in       1068376 non-null  int64  
 2   bytes_out      1068376 non-null  int64  
 3   dest_ip        1068376 non-null  int64  
 4   dest_port      964168 non-null   float64
 5   entropy        1068376 non-null  float64
 6   num_pkts_out   1068376 non-null  int64  
 7   num_pkts_in    1068376 non-null  int64  
 8   proto          1068376 non-null  int64  
 9   src_ip         1068376 non-null  int64  
 10  src_port       964168 non-null   float64
 11  time_end       1068376 non-null  int64  
 12  time_start     1068376 non-null  int64  
 13  total_entropy  1068376 non-null  float64
 14  label          1068376 non-null  object 
 15  duration       1068376 non-null  float64
dtypes: float64(6), int64(9), object(1)
memory usage: 130.4

In [ ]:
df_dataset = df_dataset.drop(['num_pkts_out', 'num_pkts_in'], axis=1)

In [ ]:
# replace +ve and -ve infinity with NaN
df_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

# drop missing values
df_dataset.dropna(inplace=True)

print(df_dataset.duplicated().sum())

4447


In [ ]:
df_dataset.drop_duplicates(inplace = True)
print(df_dataset.duplicated().sum())

0


In [ ]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 959721 entries, 0 to 1068374
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        959721 non-null  float64
 1   bytes_in       959721 non-null  int64  
 2   bytes_out      959721 non-null  int64  
 3   dest_ip        959721 non-null  int64  
 4   dest_port      959721 non-null  float64
 5   entropy        959721 non-null  float64
 6   proto          959721 non-null  int64  
 7   src_ip         959721 non-null  int64  
 8   src_port       959721 non-null  float64
 9   time_end       959721 non-null  int64  
 10  time_start     959721 non-null  int64  
 11  total_entropy  959721 non-null  float64
 12  label          959721 non-null  object 
 13  duration       959721 non-null  float64
dtypes: float64(6), int64(7), object(1)
memory usage: 109.8+ MB


In [ ]:
df_dataset["label"].value_counts()

,count
label,
benign,516220
outlier,365385
malicious,78116


In [ ]:
old_value = 'benign'
new_value = 0
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

old_value = 'outlier'
new_value = 2
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

old_value = 'malicious'
new_value = 1
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)



In [ ]:
df_dataset = df_dataset[df_dataset.label != 2]

In [ ]:
df_dataset["label"].value_counts()

,count
label,
0,516220
1,78116


In [ ]:
df=df_dataset
train, test = train_test_split(df, test_size=0.2)
train.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/train.csv', index=False)
test.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/test.csv', index=False)

==========================================


#           Start From Here


==========================================

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/MITM/LLFlow-v3/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MITM/LLFlow-v3/test.csv')

In [ ]:
min_max_scaler = MinMaxScaler().fit(train[['avg_ipt', 'bytes_in', 'bytes_out', 'dest_port', 'entropy','src_port',
                                           'total_entropy', 'duration']])
numerical_columns = ['avg_ipt', 'bytes_in', 'bytes_out', 'dest_port', 'entropy', 'src_port',
                    'total_entropy', 'duration']

train[numerical_columns] = min_max_scaler.transform(train[numerical_columns])
test[numerical_columns] = min_max_scaler.transform(test[numerical_columns])

In [ ]:
#print("Full dataset:\n")
#print("Benign: " + str(df["label"].value_counts()[[2]].sum()))
#print("Malicious: " + str(df["label"].value_counts()[[1]].sum()))
#print("outlier: " + str(df["label"].value_counts()[[0]].sum()))
#print("---------------")

print("Training set:\n")
print("Benign: " + str(train["label"].value_counts()[[0]].sum()))
#print("Malicious: " + str(train["label"].value_counts()[[2]].sum()))
print("outlier: " + str(train["label"].value_counts()[[1]].sum()))
print("---------------")

print("Test set:\n")
print("Benign: " + str(test["label"].value_counts()[[0]].sum()))
#print("Malicious: " + str(test["label"].value_counts()[[2]].sum()))
print("outlier: " + str(test["label"].value_counts()[[1]].sum()))

Training set:

Benign: 412898
outlier: 62570
---------------
Test set:

Benign: 103322
outlier: 15546


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475468 entries, 0 to 475467
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        475468 non-null  float64
 1   bytes_in       475468 non-null  float64
 2   bytes_out      475468 non-null  float64
 3   dest_ip        475468 non-null  int64  
 4   dest_port      475468 non-null  float64
 5   entropy        475468 non-null  float64
 6   proto          475468 non-null  int64  
 7   src_ip         475468 non-null  int64  
 8   src_port       475468 non-null  float64
 9   time_end       475468 non-null  int64  
 10  time_start     475468 non-null  int64  
 11  total_entropy  475468 non-null  float64
 12  label          475468 non-null  int64  
 13  duration       475468 non-null  float64
dtypes: float64(8), int64(6)
memory usage: 50.8 MB


In [ ]:
train.drop(['time_start'], axis=1, inplace=True)
test.drop(['time_start'], axis=1, inplace=True)
train.drop(['time_end'], axis=1, inplace=True)
test.drop(['time_end'], axis=1, inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475468 entries, 0 to 475467
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        475468 non-null  float64
 1   bytes_in       475468 non-null  float64
 2   bytes_out      475468 non-null  float64
 3   dest_ip        475468 non-null  int64  
 4   dest_port      475468 non-null  float64
 5   entropy        475468 non-null  float64
 6   proto          475468 non-null  int64  
 7   src_ip         475468 non-null  int64  
 8   src_port       475468 non-null  float64
 9   total_entropy  475468 non-null  float64
 10  label          475468 non-null  int64  
 11  duration       475468 non-null  float64
dtypes: float64(8), int64(4)
memory usage: 43.5 MB


In [ ]:
y_train = np.array(train.pop('label'))# pop removes "Label" from the dataframe
X_train = train.values
y_test = np.array(test.pop("label")) # pop removes "Label" from the dataframe
X_test = test.values

In [ ]:
results = y_test
df_results = pd.DataFrame(results)
df_results.head()

,0
0,0
1,0
2,0
3,0
4,0


In [ ]:
acc_target = 0.95 ## Accuracy level

**Decision Tree Classifier**

```
```



In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', max_depth=5)

parameters = {'max_depth': randint(2, 10), 'min_samples_split': randint(2, 10), 'min_samples_leaf': [1, 2, 3, 4, 5], 'criterion': ['gini', 'entropy']}

random_search = RandomizedSearchCV(dt, param_distributions=parameters, n_iter=20, cv=5, n_jobs=-1)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=DecisionTreeClassifier(criterion='entropy',
                                                    max_depth=5),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7dbf4af54130>,
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7dbf4b48d090>})

In [ ]:
from sklearn import metrics
# dt = DecisionTreeClassifier(**random_search.best_params_)
dt.fit(X_train,y_train)

# Predicting Train & Test Results
y_dt = dt.predict(X_train)
y_pred_dt = dt.predict(X_test)
df_y_pred_dt = pd.DataFrame(y_pred_dt)

# Accuracy
print("Accuracy on Training set: ",metrics.accuracy_score(y_train, y_dt))
print("Accuracy on Testing set: ",metrics.accuracy_score(y_test, y_pred_dt))

Accuracy on Training set:  0.9900519067529255
Accuracy on Testing set:  0.9900982602550729


In [ ]:
if metrics.accuracy_score(y_test, y_pred_dt) >= acc_target :
  frames = [df_results,df_y_pred_dt]
  df_results = pd.concat(frames, axis=1)
df_results.head()

,0,0
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [ ]:

df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)


In [ ]:
df_y_test = pd.DataFrame(y_test)
df_y_test.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/y_test.csv', index=False)

In [ ]:
df_ydt = pd.DataFrame(y_dt)
df_ydt.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_rain_dt.csv', index=False)
df_y_pred_dt = pd.DataFrame(y_pred_dt)
df_y_pred_dt.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_test_dt.csv', index=False)
df_y_test = pd.DataFrame(y_test)
df_y_test.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/y_test.csv', index=False)


**# Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)

parameters = {'n_estimators': randint(50, 100), 'max_depth': randint(2, 8), 'min_samples_split': randint(2, 10),
              'min_samples_leaf': randint(1, 10), 'max_features': ['sqrt', 'log2', None]}

rf.fit(X_train, y_train)

# Predicting Train & Test Results
y_rf = rf.predict(X_train)
y_pred_rf = rf.predict(X_test)

# Accuracy
print("Accuracy on Training set: ",metrics.accuracy_score(y_train, y_rf))
print("Accuracy on Testing set: ",metrics.accuracy_score(y_test, y_pred_rf))

Accuracy on Training set:  0.9999726585175027
Accuracy on Testing set:  0.9997139684355756


In [ ]:
df_y_pred_rf = pd.DataFrame(y_pred_rf)
if metrics.accuracy_score(y_test, y_pred_rf) >= acc_target :
  frames = [df_results,df_y_pred_rf]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)
df_results.head()

,0,0,0
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [ ]:
df_yrf = pd.DataFrame(y_rf)
df_yrf.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_train_rf.csv', index=False)

df_y_pred_rf.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_test_rf.csv', index=False)


===========================================================

**GRU**


===========================================================


In [ ]:


# One-hot encode the target variable for multiclass classification
encoder_gru = OneHotEncoder()
y_train_encoded = encoder_gru.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_encoded = encoder_gru.transform(y_test.reshape(-1, 1)).toarray()

hidden_units = 32
X_train_reshaped = X_train.reshape(-1, 11, 1)  # Reshape X_train to (batch_size, sequence_length, num_features)

# Create the GRU model
num_classes = len(np.unique(y_train))
model_gru = Sequential()
model_gru.add(GRU(32, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))  # GRU layer with 32 unitsmodel.add(Dense(1, activation='sigmoid'))
model_gru.add(Dense(num_classes, activation='softmax'))
model_gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_gru.fit(X_train, y_train_encoded, epochs=12, batch_size=32, validation_data=(X_test, y_test_encoded))



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 123s 8ms/step - accuracy: 0.9234 - loss: 0.2183 - val_accuracy: 0.9942 - val_loss: 0.0459
Epoch 2/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 141s 8ms/step - accuracy: 0.9929 - loss: 0.0284 - val_accuracy: 0.9951 - val_loss: 0.0195
Epoch 3/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 114s 8ms/step - accuracy: 0.9946 - loss: 0.0221 - val_accuracy: 0.9962 - val_loss: 0.0190
Epoch 4/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 169s 9ms/step - accuracy: 0.9957 - loss: 0.0176 - val_accuracy: 0.9967 - val_loss: 0.0136
Epoch 5/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 129s 9ms/step - accuracy: 0.9963 - loss: 0.0154 - val_accuracy: 0.9975 - val_loss: 0.0124
Epoch 6/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 129s 8ms/step - accuracy: 0.9966 - loss: 0.0139 - val_accuracy: 0.9968 - val_loss: 0.0129
Epoch 7/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 116s 8ms/step - accuracy: 0.9969 - loss: 0.0129 - val_accuracy: 0.9975 - val_loss: 0.0104
Epoch 8/12
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 140s 8ms/step - ac

In [ ]:
# Predict labels for test data

#y_Pred_Train_GRU= model.predict_classes(X_train)
#y_pred_Test_GRU= model.predict_classes(X_test)
#y_Pred_Train_GRU = y_Pred_Train_GRU.flatten()
#y_pred_Test_GRU = y_pred_Test_GRU.flatten()

y_Pred_Train_NN_GRU= model_gru.predict(X_train)
y_pred_Test_NN_GRU= model_gru.predict(X_test)

df_Pred_Train_NN_GRU = pd.DataFrame(y_Pred_Train_NN_GRU)
df_Pred_Train_NN_GRU.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_GRU.csv', index=False)
df_pred_Test_NN_GRU = pd.DataFrame(y_pred_Test_NN_GRU)
df_pred_Test_NN_GRU.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_GRU.csv', index=False)



14859/14859 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step
3715/3715 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step


In [ ]:
y_Pred_Train_NN_GRU_argmax = np.argmax(y_Pred_Train_NN_GRU, axis=1)
y_pred_Test_NN_GRU_argmax = np.argmax(y_pred_Test_NN_GRU, axis=1)



In [ ]:

# Evaluate the model
accuracy_gru = accuracy_score(y_test, y_pred_Test_NN_GRU_argmax)
print('Accuracy: %.2f' % (accuracy_gru * 100))

Accuracy: 99.78


In [ ]:
df_pred_Test_NN_GRU_argmax = pd.DataFrame(y_pred_Test_NN_GRU_argmax)
if accuracy_gru >= acc_target :
  frames = [df_results,df_pred_Test_NN_GRU_argmax]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)

In [ ]:
df_y_Pred_Train_NN_GRU_argmax = pd.DataFrame(y_Pred_Train_NN_GRU_argmax)
df_y_Pred_Train_NN_GRU_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_GRU_argmax.csv', index=False)
df_pred_Test_NN_GRU_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_GRU_argmax.csv', index=False)

===============================================

#           **NN**

=================================================

In [ ]:
model_nn = Sequential() # initializing model
#lstm
model_nn.add(Conv1D(512,1, padding="same",input_shape = (X_train.shape[1], 1), activation='relu'))
model_nn.add(MaxPool1D(pool_size=(2)))
model_nn.add(LSTM(units=16,return_sequences=False,dropout=0.2))
model_nn.add(Dense(units=3,activation='softmax'))


model_nn.compile(loss=SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model_nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 11, 512)             │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 5, 512)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 16)                  │          33,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,931 (136.45 KB)

 Trainable params: 34,931 (136.45 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history=model_nn.fit(X_train, y_train, epochs=15, batch_size=512, validation_data=(X_test,y_test))

Epoch 1/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 62s 58ms/step - accuracy: 0.8595 - loss: 0.4262 - val_accuracy: 0.8692 - val_loss: 0.3775
Epoch 2/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 84s 60ms/step - accuracy: 0.8915 - loss: 0.3193 - val_accuracy: 0.9417 - val_loss: 0.1917
Epoch 3/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 84s 63ms/step - accuracy: 0.9488 - loss: 0.1582 - val_accuracy: 0.9792 - val_loss: 0.0652
Epoch 4/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 57s 62ms/step - accuracy: 0.9807 - loss: 0.0643 - val_accuracy: 0.9849 - val_loss: 0.0476
Epoch 5/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 55s 59ms/step - accuracy: 0.9847 - loss: 0.0498 - val_accuracy: 0.9869 - val_loss: 0.0394
Epoch 6/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 83s 61ms/step - accuracy: 0.9872 - loss: 0.0426 - val_accuracy: 0.9876 - val_loss: 0.0394
Epoch 7/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 59s 63ms/step - accuracy: 0.9886 - loss: 0.0392 - val_accuracy: 0.9903 - val_loss: 0.0332
Epoch 8/15
929/929 ━━━━━━━━━━━━━━━━━━━━ 54s 58ms/step - accuracy: 0.9898 - loss: 0.0356 - 

In [ ]:
# Predicting Train & Test Results
y_Pred_Train_NN_LSTM = model_nn.predict(X_train)
y_pred_Test_NN_LSTM = model_nn.predict(X_test)

14859/14859 ━━━━━━━━━━━━━━━━━━━━ 45s 3ms/step
3715/3715 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step


In [ ]:
y_Pred_Train_NN_LSTM_argmax = np.argmax(y_Pred_Train_NN_LSTM, axis=1)
y_pred_Test_NN_LSTM_argmax = np.argmax(y_pred_Test_NN_LSTM, axis=1)

In [ ]:
# Evaluate the model
accuracy_nn = accuracy_score(y_test, y_pred_Test_NN_LSTM_argmax)
print('Accuracy: %.2f' % (accuracy_gru * 100))

Accuracy: 99.78


In [ ]:
df_pred_Test_NN_LSTM_argmax = pd.DataFrame(y_pred_Test_NN_LSTM_argmax)

if accuracy_nn >= acc_target :
  frames = [df_results,df_pred_Test_NN_LSTM_argmax]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)
df_results.head()

,0,0,0,0,0
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [ ]:
df_Pred_Train_NN_LSTM = pd.DataFrame(y_Pred_Train_NN_LSTM)
df_Pred_Train_NN_LSTM.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_LSTM.csv', index=False)
df_pred_Test_NN_LSTM = pd.DataFrame(y_pred_Test_NN_LSTM)
df_pred_Test_NN_LSTM.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_LSTM.csv', index=False)
df_Pred_Train_NN_LSTM_argmax = pd.DataFrame(y_Pred_Train_NN_LSTM_argmax)
df_Pred_Train_NN_LSTM_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_LSTM_argmax.csv', index=False)
df_pred_Test_NN_LSTM_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_LSTM_argmax.csv', index=False)

=========================================

**XGBOOST**

=========================================

In [ ]:
#convert the features

y_train_categ = to_categorical(y_train)
y_test_categ = to_categorical(y_test)

In [ ]:


xgb_model = xgb.XGBRegressor(enable_categorical=True)
xgb_model.fit(X_train, y_train_categ)
y_test_xgb_pred = xgb_model.predict(X_test)
y_train_xgb_pred = xgb_model.predict(X_train)
y_test_xgb_pred_argmax = np.argmax(y_test_xgb_pred, axis=1)
y_train_xgb_pred_argmax = np.argmax(y_train_xgb_pred, axis=1)

In [ ]:
# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_test_xgb_pred_argmax)
print('Accuracy: %.2f' % (accuracy_xgb * 100))

Accuracy: 99.95


In [ ]:
df_pred_Test_xgb_argmax = pd.DataFrame(y_test_xgb_pred_argmax)

if accuracy_xgb >= acc_target :
  frames = [df_results,df_pred_Test_xgb_argmax]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)
df_results.head()

,0,0,0,0,0,0
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [ ]:
df_pred_Test_XGB = pd.DataFrame(y_test_xgb_pred)
df_pred_Test_XGB.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_XGB.csv', index=False)
df_pred_Train_XGB = pd.DataFrame(y_train_xgb_pred)
df_pred_Train_XGB.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_XGB.csv', index=False)
df_pred_Test_XGB_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_XGB_argmax.csv', index=False)
df_pred_Train_XGB_argmax = pd.DataFrame(y_train_xgb_pred_argmax)
df_pred_Train_XGB_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_XGB_argmax.csv', index=False)

NameError: name 'df_pred_Test_XGB_argmax' is not defined


===========================================================================

**SVM**

===========================================================================


In [ ]:

X_train = np.delete(X_train, np.where(train.columns == 'src_ip')[0][0], axis=1) # Get the index of the 'src_ip' column and delete it
X_train = np.delete(X_train, np.where(train.columns == 'dest_ip')[0][0], axis=1) # Get the index of the 'dest_ip' column and delete it
X_test = np.delete(X_test, np.where(train.columns == 'src_ip')[0][0], axis=1) # Get the index of the 'src_ip' column and delete it
X_test = np.delete(X_test, np.where(train.columns == 'dest_ip')[0][0], axis=1) # Get the index of the 'dest_ip' column and delete it



In [ ]:
svm_model = SVC(kernel='linear', C=1.0)  # Using a linear kernel with default parameters
svm_model.fit(X_train, y_train)

y_test_svm_pred = svm_model.predict(X_test)
y_train_svm_pred = svm_model.predict(X_train)


df_pred_Test_SVM = pd.DataFrame(y_test_svm_pred)
df_pred_Test_SVM.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_SVM.csv', index=False)
df_pred_Train_SVM = pd.DataFrame(y_train_svm_pred)
df_pred_Train_SVM.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_SVM.csv', index=False)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_test_svm_pred_argmax)
print(f"Accuracy: {accuracy}")

NameError: name 'y_test_svm_pred_argmax' is not defined

In [ ]:
y_test_svm_pred_argmax = np.argmax(y_test_svm_pred, axis=1)
y_train_svm_pred_argmax = np.argmax(y_train_svm_pred, axis=1)

df_pred_Test_SVM_argmax = pd.DataFrame(y_test_svm_pred_argmax)
df_pred_Test_SVM_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_SVM_argmax.csv', index=False)
df_pred_Train_SVM_argmax = pd.DataFrame(y_train_svm_pred_argmax)
df_pred_Train_SVM_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_SVM_argmax.csv', index=False)

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_test_svm_pred_argmax)
print(f"Accuracy: {accuracy}")


================================================================

**Logistic Regression**

 ===============================================================

In [ ]:
log_reg_model = LogisticRegression(solver='sag', max_iter=10000)
log_reg_model.fit(X_train, y_train)

y_test_log_reg_pred = log_reg_model.predict(X_test)
y_train_log_reg_pred = log_reg_model.predict(X_train)

# Calculate the accuracy of the model
accuracy_lr = accuracy_score(y_test, y_test_log_reg_pred)
print(f"Accuracy: {accuracy_lr * 100}")

Accuracy: 92.3789413467039


In [ ]:
df_pred_Test_log_reg = pd.DataFrame(y_test_log_reg_pred)


if accuracy_lr >= acc_target :
  frames = [df_results,df_pred_Test_log_reg]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)
df_results.head()

,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [ ]:
df_pred_Test_log_reg = pd.DataFrame(y_test_log_reg_pred)

df_pred_Test_log_reg.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_log_reg1.csv', index=False)
df_pred_Train_log_reg = pd.DataFrame(y_train_log_reg_pred)
df_pred_Train_log_reg.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_log_reg1.csv', index=False)

In [ ]:

y_train_log_reg_pred_argmax = np.argmax(y_train_log_reg_pred, axis=1)


df_pred_Test_log_reg_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_log_reg_argmax.csv', index=False)
df_pred_Train_log_reg_argmax = pd.DataFrame(y_train_log_reg_pred_argmax)
df_pred_Train_log_reg_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_log_reg_argmax.csv', index=False)


====================================================================

**KNN**

====================================================================


In [ ]:
# Define the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=3)  # Initialize the k-NN classifier with k=3

# Train the classifier
knn.fit(X_train, y_train)

# Make predictions on the test set
y_test_knn_pred = knn.predict(X_test)
y_train_knn_pred = knn.predict(X_train)

# Calculate the accuracy of the model
accuracy_knn = accuracy_score(y_test, y_test_knn_pred)
print(f"Accuracy: {accuracy_knn}")


Accuracy: 0.9990577783760137


In [ ]:
df_pred_Test_knn = pd.DataFrame(y_test_knn_pred)

if accuracy_knn >= acc_target :
  frames = [df_results,df_pred_Test_knn]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)

In [ ]:
df_pred_Test_knn.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_knn.csv', index=False)
df_pred_Train_knn = pd.DataFrame(y_train_knn_pred)
df_pred_Train_knn.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_knn.csv', index=False)


=========================================================================

 **Naive Bayes**

 ========================================================================


In [ ]:
X_train = np.delete(X_train, np.where(train.columns == 'src_ip')[0][0], axis=1) # Get the index of the 'src_ip' column and delete it
X_train = np.delete(X_train, np.where(train.columns == 'dest_ip')[0][0], axis=1) # Get the index of the 'dest_ip' column and delete it
X_test = np.delete(X_test, np.where(train.columns == 'src_ip')[0][0], axis=1) # Get the index of the 'src_ip' column and delete it
X_test = np.delete(X_test, np.where(train.columns == 'dest_ip')[0][0], axis=1) # Get the index of the 'dest_ip' column and delete it

In [ ]:

from sklearn.naive_bayes import ComplementNB

# Define the Naive Bayes classifier
nb_classifier = ComplementNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_test_NvB_pred = nb_classifier.predict(X_test)
y_train_NvB_pred = nb_classifier.predict(X_train)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_test_NvB_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5425095730547813


In [ ]:
df_pred_Test_NvB = pd.DataFrame(y_test_NvB_pred)
df_pred_Test_NvB.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NvB.csv', index=False)
df_pred_Train_NvB = pd.DataFrame(y_train_NvB_pred)
df_pred_Train_NvB.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_NvB.csv', index=False)


==================================================================

 **AdaBoost**

 =================================================================


In [ ]:
# Define the AdaBoost classifier with a Decision Tree base estimator
adaboost_classifier = AdaBoostClassifier(n_estimators=50, random_state=42)  # 50 weak learners

# Train the classifier
adaboost_classifier.fit(X_train, y_train)



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning:

The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.



AdaBoostClassifier(random_state=42)

In [ ]:
# Make predictions on the test set
y_test_AdaB_pred = adaboost_classifier.predict(X_test)
y_train_AdaB_pred = adaboost_classifier.predict(X_train)

# Calculate the accuracy of the model
accuracy_AdaB = accuracy_score(y_test, y_test_AdaB_pred)
print(f"Accuracy: {accuracy_AdaB}")

Accuracy: 0.9971733351280412


In [ ]:
df_pred_Test_AdaB = pd.DataFrame(y_test_AdaB_pred)

if accuracy_AdaB >= acc_target :
  frames = [df_results,df_pred_Test_AdaB]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)
df_results.head()

,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [ ]:

df_pred_Test_AdaB.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_AdaB.csv', index=False)
df_pred_Train_AdaB = pd.DataFrame(y_train_AdaB_pred)
df_pred_Train_AdaB.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_AdaB.csv', index=False)


=================================================================

**Catboost**

==================================================================


In [ ]:
# Define the CatBoost classifier for multiclass classification
catboost_classifier = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=2, loss_function='MultiClass')

# Train the classifier
train_pool = Pool(data=X_train, label=y_train)
test_pool = Pool(data=X_test, label=y_test)

catboost_classifier.fit(train_pool, eval_set=test_pool, verbose=False)


In [ ]:
# Make predictions on the test set
y_test_catb_pred = catboost_classifier.predict(X_test)
y_train_catb_pred = catboost_classifier.predict(X_train)

# Calculate the accuracy of the model
accuracy_catb = accuracy_score(y_test, y_test_catb_pred)
print(f"Accuracy: {accuracy_catb}")

Accuracy: 0.9981155567520275


In [ ]:
df_pred_Test_catb = pd.DataFrame(y_test_catb_pred)

if accuracy_catb >= acc_target :
  frames = [df_results,df_pred_Test_catb]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)
df_results.head()

,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0


In [ ]:

df_pred_Test_catb.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_CatB.csv', index=False)
df_pred_Train_catb = pd.DataFrame(y_train_catb_pred)
df_pred_Train_catb.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_CatB.csv', index=False)


===================================================================

 **Convolutional Neural Networks (CNN)**

 ===================================================================




In [ ]:


# Standardize the input data
scaler_cnn = StandardScaler()
X_train_scaler = scaler_cnn.fit_transform(X_train)
X_test_scaler = scaler_cnn.transform(X_test)

# Define a simple Multi-Layer Perceptron (MLP) model
model = Sequential([
    Dense(128, activation='relu', input_shape=(11,)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Assuming 10 classes for classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaler, y_train, epochs=25, batch_size=32, validation_data=(X_test_scaler, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.9742 - loss: 0.0947 - val_accuracy: 0.9927 - val_loss: 0.0265
Epoch 2/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9938 - loss: 0.0227 - val_accuracy: 0.9957 - val_loss: 0.0150
Epoch 3/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9953 - loss: 0.0176 - val_accuracy: 0.9965 - val_loss: 0.0158
Epoch 4/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9959 - loss: 0.0156 - val_accuracy: 0.9963 - val_loss: 0.0132
Epoch 5/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.9962 - loss: 0.0150 - val_accuracy: 0.9968 - val_loss: 0.0113
Epoch 6/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - accuracy: 0.9962 - loss: 0.0140 - val_accuracy: 0.9973 - val_loss: 0.0113
Epoch 7/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9965 - loss: 0.0133 - val_accuracy: 0.9966 - val_loss: 0.0128
Epoch 8/25
14859/14859 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - accuracy: 

In [ ]:
y_Pred_Train_NN_CNN = model.predict(X_train_scaler)
y_pred_Test_NN_CNN = model.predict(X_test_scaler)

In [ ]:
y_Pred_Train_NN_CNN_argmax = np.argmax(y_Pred_Train_NN_CNN, axis=1)
y_pred_Test_NN_CNN_argmax = np.argmax(y_pred_Test_NN_CNN, axis=1)

In [ ]:
# Evaluate the model
accuracy_cnn = accuracy_score(y_test, y_pred_Test_NN_CNN_argmax)
print('Accuracy: %.2f' % (accuracy_cnn * 100))

In [ ]:
df_pred_Test_NN_CNN_argmax = pd.DataFrame(y_pred_Test_NN_CNN_argmax)

if accuracy_cnn >= acc_target :
  frames = [df_results,df_pred_Test_NN_CNN_argmax]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)

In [ ]:
df_Pred_Train_NN_CNN = pd.DataFrame(y_Pred_Train_NN_CNN)
df_Pred_Train_NN_CNN.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_CNN.csv', index=False)
df_pred_Test_NN_CNN = pd.DataFrame(y_pred_Test_NN_CNN)
df_pred_Test_NN_CNN.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_CNN.csv', index=False)
df_Pred_Train_NN_CNN_argmax = pd.DataFrame(y_Pred_Train_NN_CNN_argmax)
df_Pred_Train_NN_CNN_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_CNN_argmax.csv', index=False)

df_pred_Test_NN_CNN_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_CNN_argmax.csv', index=False)


===============================================================

**Recurrent Neural Networks (RNN)**

===============================================================


In [ ]:
# Reshape the data for RNN input - considering each row as a sequence
X_train_reshape = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshape = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define an RNN model with LSTM cells
model_rnn = Sequential([
    LSTM(64, input_shape=(1, 11)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model_rnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_rnn.fit(X_train_reshape, y_train, epochs=25, batch_size=32, validation_data=(X_test_reshape, y_test))

Epoch 1/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.9361 - loss: 0.2035 - val_accuracy: 0.9560 - val_loss: 0.1279
Epoch 2/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 71s 3ms/step - accuracy: 0.9571 - loss: 0.1239 - val_accuracy: 0.9596 - val_loss: 0.1145
Epoch 3/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 69s 3ms/step - accuracy: 0.9594 - loss: 0.1139 - val_accuracy: 0.9601 - val_loss: 0.1164
Epoch 4/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.9612 - loss: 0.1084 - val_accuracy: 0.9638 - val_loss: 0.1040
Epoch 5/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 81s 3ms/step - accuracy: 0.9622 - loss: 0.1056 - val_accuracy: 0.9636 - val_loss: 0.1069
Epoch 6/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.9631 - loss: 0.1034 - val_accuracy: 0.9632 - val_loss: 0.1011
Epoch 7/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 84s 3ms/step - accuracy: 0.9639 - loss: 0.1011 - val_accuracy: 0.9648 - val_loss: 0.0989
Epoch 8/25
23993/23993 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - accuracy: 

KeyboardInterrupt: 

In [ ]:
y_Pred_Train_NN_RNN = model.predict(X_train_reshape)
y_pred_Test_NN_RNN = model.predict(X_test_reshape)

df_Pred_Train_NN_RNN = pd.DataFrame(y_Pred_Train_NN_RNN)
df_Pred_Train_NN_RNN.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_RNN.csv', index=False)
df_pred_Test_NN_RNN = pd.DataFrame(y_pred_Test_NN_RNN)
df_pred_Test_NN_RNN.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_RNN.csv', index=False)



14859/14859 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step
3715/3715 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


In [ ]:
y_Pred_Train_NN_RNN_argmax = np.argmax(y_Pred_Train_NN_RNN, axis=1)
y_pred_Test_NN_RNN_argmax = np.argmax(y_pred_Test_NN_RNN, axis=1)

In [ ]:
# Evaluate the model
accuracy_rnn = accuracy_score(y_test, y_pred_Test_NN_RNN_argmax)
print('Accuracy: %.2f' % (accuracy_rnn * 100))

In [ ]:
df_pred_Test_NN_RNN_argmax = pd.DataFrame(y_pred_Test_NN_RNN_argmax)


if accuracy_rnn >= acc_target :
  frames = [df_results,df_pred_Test_NN_RNN_argmax]
  df_results = pd.concat(frames, axis=1)
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)

In [ ]:


df_Pred_Train_NN_RNN_argmax = pd.DataFrame(y_Pred_Train_NN_RNN_argmax)
df_Pred_Train_NN_RNN_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/Pred_Train_NN_RNN_argmax.csv', index=False)

df_pred_Test_NN_RNN_argmax.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_NN_RNN_argmax.csv', index=False)


==============================================================

**Quadratic Discriminant Analysis (QDA)**

==============================================================



In [ ]:
# Initialize the QDA model
qda = QuadraticDiscriminantAnalysis()

# Train the QDA model
qda.fit(X_train, y_train)

# Make predictions on the test set
y_test_qda_pred = qda.predict(X_test)
y_train_qda_pred = qda.predict(X_train)

# Calculate the accuracy of the model
accuracy_qda = accuracy_score(y_test, y_test_qda_pred)
print(f"Accuracy: {accuracy_qda}")

/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:935: UserWarning:

Variables are collinear



Accuracy: 0.6309880434499466


In [ ]:
df_pred_Test_qda = pd.DataFrame(y_test_qda_pred)

if accuracy_qda >= acc_target :
  frames = [df_results,df_pred_Test_qda]
  df_results = pd.concat(frames, axis=1)
  print "QDA Pass"
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)

In [ ]:
df_pred_Test_qda.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Test_qda.csv', index=False)
df_pred_Train_qda = pd.DataFrame(y_train_qda_pred)
df_pred_Train_qda.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/pred_Train_qda.csv', index=False)

==============================================================

**Linear Discriminant Analysis (LDA)**

==============================================================

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Initialize the LDA model
lda = LinearDiscriminantAnalysis()

# Train the LDA model
lda.fit(X_train, y_train)

# Make predictions on the test set
y_test_lda_pred = lda.predict(X_test)
y_train_lda_pred = lda.predict(X_train)

# Calculate the accuracy of the model
accuracy_lda = accuracy_score(y_test, y_test_lda_pred)
print(f"Accuracy: {accuracy_lda}")

Accuracy: 0.8050743702623148


In [ ]:
df_pred_Test_qda = pd.DataFrame(y_test_lda_pred)

if accuracy_lda >= acc_target :
  frames = [df_results,df_pred_Test_qda]
  df_results = pd.concat(frames, axis=1)
  print "LDA Pass"
df_results.to_csv('/content/drive/MyDrive/MITM/LLFlow-v3/results.csv', index=False)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=5000, random_state=42)

# Train the MLP model
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_test_mlp_pred = mlp.predict(X_test)
y_train_mlp_pred = mlp.predict(X_train)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_test_mlp_pred)
print(f"Accuracy: {accuracy}")